# Pinecone Vector DB 

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## 1. 모델명, 제품명, 수리부품, 수리비용 검색

### 파일 로드

In [3]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./AS_product.csv', encoding = 'utf-8')
documents = loader.load()
print(len(documents))
documents[0]


95


Document(metadata={'source': './AS_product.csv', 'row': 0}, page_content='\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 디스플레이(메인)\n수리비용: 543,000')

### 임베딩 모델 생성


In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')


### Pinecone

In [5]:
# Pinecone 클라이언트
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')


In [6]:
# 클라이언트 객체
vector_db = PineconeVectorStore(
    embedding=embeddings,
    index_name = PINECONE_INDEX_NAME,
    namespace = PINECONE_NAMESPACE,
    pinecone_api_key = PINECONE_API_KEY
)

results = vector_db.similarity_search(
    '갤럭시 탭 S10 울트라 화면이 고장났어요.',
    k = 5,
    namespace = PINECONE_NAMESPACE)  # 검색의 논리적인 단위
results


[Document(id='d613cf36-2c49-42f5-990e-7addbe60ed84', metadata={'row': 72.0, 'source': './AS_product.csv'}, page_content='\ufeff모델 종류: 탭\n모델명: SM-X920NZAAKOO\n제품명: 갤럭시_탭_S10_ULTRA(울트라)_WI-FI\n수리부품: 후면케이스\n수리비용: 284,000'),
 Document(id='78a08219-ef82-4f21-8477-d29830822e9b', metadata={'row': 68.0, 'source': './AS_product.csv'}, page_content='\ufeff모델 종류: 탭\n모델명: SM-X920NZAAKOO\n제품명: 갤럭시_탭_S10_ULTRA(울트라)_WI-FI\n수리부품: 디스플레이\n수리비용: 585,000'),
 Document(id='404c9ff9-3be5-45f7-a251-502f69d3e0ac', metadata={'row': 71.0, 'source': './AS_product.csv'}, page_content='\ufeff모델 종류: 탭\n모델명: SM-X920NZAAKOO\n제품명: 갤럭시_탭_S10_ULTRA(울트라)_WI-FI\n수리부품: 카메라\n수리비용: 72,000'),
 Document(id='95c6ec86-f55c-4d1b-b89a-ae117dcee760', metadata={'row': 70.0, 'source': './AS_product.csv'}, page_content='\ufeff모델 종류: 탭\n모델명: SM-X920NZAAKOO\n제품명: 갤럭시_탭_S10_ULTRA(울트라)_WI-FI\n수리부품: 배터리\n수리비용: 88,500'),
 Document(id='b20c4e67-de69-448f-86da-87db75e29f5c', metadata={'row': 77.0, 'source': './AS_product.csv'}, page_content='\

In [16]:
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

def repair_product_information(query):
    """
    :param query: 전자제품 제품명 + 수리품목 텍스트 
    :return: 
    """
    embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')
    # 클라이언트 객체
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name = PINECONE_INDEX_NAME,
        namespace = PINECONE_NAMESPACE,
        pinecone_api_key = PINECONE_API_KEY
    )
    # 유사도 검색
    results = vector_db.similarity_search(
    query,
    k = 10,
    namespace = PINECONE_NAMESPACE
    ) 
    a='\n'.join([doc.page_content for doc in results])
    print(a)
    return {
        'as_details' : query,
        'repair_information' : '\n'.join([doc.page_content for doc in results])
    }


In [19]:
repair_product_information('갤럭시 탭 S10 울트라 화면이 고장났어요.')


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [9]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(repair_product_information)
input_data = {'text': '갤럭시Z플립3의 배터리가 망가졌어요.'}
chain.invoke(input_data['text'])


{'as_details': '갤럭시Z플립3의 배터리가 망가졌어요.',
 'repair_information': '\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 배터리\n수리비용: 96,500\n\ufeff모델 종류: 스마트폰\n모델명: SM-F741NZWAKOO\n제품명: 갤럭시_Z_플립6\n수리부품: 배터리\n수리비용: 130,000\n\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 디스플레이(메인)\n수리비용: 543,000\n\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 디스플레이(서브)\n수리비용: 130,000\n\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 후면케이스\n수리비용: 33,000\n\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 카메라\n수리비용: 64,000\n\ufeff모델 종류: 스마트폰\n모델명: SM-F956NAKEKOO\n제품명: 갤럭시_Z_폴드6\n수리부품: 배터리\n수리비용: 111,500\n\ufeff모델 종류: 스마트폰\n모델명: SM-S921NZYEKOO\n제품명: 갤럭시_S24\n수리부품: 배터리\n수리비용: 67,500\n\ufeff모델 종류: 스마트폰\n모델명: SM-F711N1BFKOO\n제품명: 갤럭시_Z_플립3\n수리부품: 메인보드\n수리비용: 460,000\n\ufeff모델 종류: 노트북\n모델명: NT960XMB-K\n제품명: 갤럭시_북4_EDGE\n수리부품: 배터리\n수리비용: 172,000'}

## 2. 수리부품 추천 - 수리비용 안내

In [10]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def product_repair_cost(query):
    
    prompt = ChatPromptTemplate.from_messages([
        ('system', """
Persona :
당신은 전자제품에 대한 깊은 사랑과 열정을 가진 서비스센터 직원입니다. 전자 제품에 대한 폭넓은 지식을 보유하고 있으며, 다양한 모델을 제품명만 듣고 어떤 모델 종류인지 구분해내는 능력과 고장난 문제에 따라 필요한 부품 및 수리 비용에 대해 잘 알고 있습니다. 고객이 수리 과정과 비용을 이해할 수 있도록 친근하고 다가가기 쉬운 태도로 설명하며, 초보자도 쉽게 이해할 수 있는 방식으로 설명합니다.

Role:
서비스 센터 직원으로서의 역할은 필요한 수리 부품을 쉽게 이해할 수 있는 방식으로 설명하고 수리 비용 정보를 제공하는 것입니다.  당신은 제품명을 보는 것만으로도 해당 제품이 휴대폰, TV, 노트북, 워치, 탭 중에서 어떤 모델 종류인지 구분할 수 있습니다. 고객이 제품명만 알려줘도 어떤 제품인지 파악하고, 해당하는 수리 부품과 수리 비용을 안내할 수 있어야 합니다. 고객이 최소한의 적합한 수리 부품만 사용하도록 도와 원활한 서비스를 제공합니다. 고객의 불편함을 최소화하고 적절한 부품과 비용을 추천함으로써 고객이 전자 제품을 편리하게 사용할 수 있도록 돕는 것이 목표입니다. 추측성 정보는 고객에게 혼란을 줄 수 있으므로 지양합니다.

Examples:
- 휴대폰 화면 손상 수리에 대한 예시:
고객이 화면 손상으로 인한 수리 비용 설명을 요청하는 경우, 제품의 모델명, 보증 기간, 적합한 수리 부품, 비용, 예상 수리 시간을 초등학생도 이해할 수 있도록 쉽게 설명합니다.

- "왜 TV가 작동하지 않나요?"라는 질문에 대한 예시:
TV가 작동하지 않는 데에는 여러 가지 원인이 있을 수 있습니다. 우선 전원 플러그가 제대로 꽂혀 있는지 확인해주세요. 안테나 케이블 연결이 제대로 되어있지 않거나, 케이블 방송 수신기 또는 위성 리시버의 전원이 꺼져 있을 수 있습니다. 안테나 케이블 연결 상태를 확인하거나, 케이블 방송 수신기 또는 위성 리시버의 전원을 켜주세요. 또한, 환경 조건이 영향을 줄 수 있습니다. 권장 작동 환경은 온도 10℃~40℃(50℉~104℉), 습도 10%~80%이며, 응결이 없는 환경이어야 합니다. 추가적인 문의사항이 있으시면 말씀해주세요. 감사합니다!

        """),
        HumanMessagePromptTemplate.from_template('''
        수리비용 설명에 아래의 제품명과 모델명, 수리부품, 수리비용만을 참고하여 한글로 답변해주세요.
        A/S 내용 :
        {as_details}
        수리 정보 :
        {repair_information}
        ''')
    ])
    
    model = ChatOpenAI(
        model = 'gpt-4o',
        temperature = 0.3
    )
    
    output_parser = StrOutputParser()
    
    return prompt | model | output_parser
    

## Chain 연결

In [11]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(repair_product_information) | RunnableLambda(product_repair_cost)

input_data = {'text': 'NEO QLED 8K TV 85인치의 케이블이 망가졌어요.'}
print(chain.invoke(input_data['text']))


고객님, NEO QLED 8K TV 85인치 모델(KQ85QNC850FXKR)의 케이블이 손상되셨군요. 해당 모델의 케이블 수리 비용은 33,000원입니다. 수리 진행을 원하시면 언제든지 말씀해 주세요. 추가적으로 궁금하신 점이 있으면 언제든지 문의해 주세요. 감사합니다!


In [12]:
input_data = {'text': '갤럭시Z폴드6의 카메라가 작동이 안돼요.'}
print(chain.invoke(input_data['text']))


갤럭시 Z 폴드6의 카메라가 작동하지 않는 문제에 대해 안내드리겠습니다. 해당 모델명은 SM-F956NAKEKOO로, 갤럭시 Z 폴드6에 해당합니다. 카메라 수리에 필요한 부품은 '카메라'이며, 수리 비용은 75,500원입니다. 수리 진행을 원하시면 서비스 센터를 방문해 주시기 바랍니다. 추가적인 문의사항이 있으시면 언제든지 말씀해 주세요. 감사합니다!


In [13]:
input_data = {'text': 'NEOQLED8KTV85형(인치) 디스플레이 수리비용이 얼마정도 들까요??'}
print(chain.invoke(input_data['text']))

안녕하세요! NEO QLED 8K TV 85형(인치)의 디스플레이 수리 비용에 대해 문의 주셨군요. 해당 모델명은 KQ85QNC850FXKR이며, 디스플레이 수리 비용은 2,575,000원입니다. 디스플레이는 TV의 핵심 부품 중 하나로, 수리 비용이 다소 높게 책정될 수 있습니다. 추가적인 문의사항이 있으시면 언제든지 말씀해 주세요. 감사합니다!


In [14]:
input_data = {'text': '갤럭시_북4_PRO_360 키보드를 눌러도 입력이 안돼요.'}
print(chain.invoke(input_data['text']))

갤럭시_북4_PRO_360 모델의 키보드 입력이 안 되는 문제로 수리를 원하시는군요. 해당 모델명은 NT960QGK-KD72S이며, 이 경우 필요한 수리 부품은 키보드입니다. 수리 비용은 494,000원입니다. 수리 과정에 대해 추가적인 질문이 있으시면 언제든지 말씀해 주세요. 감사합니다!


In [15]:
input_data = {'text': '갤럭시_워치6_40MM_블루투스 제품인데 충전이 잘 안되는것 같아.'}
print(chain.invoke(input_data['text']))


갤럭시 워치6 40MM 블루투스 모델(SM-R930NZEAKOO)의 충전 문제로 인해 수리가 필요하시군요. 이 경우, 배터리 교체가 필요할 수 있습니다. 해당 모델의 배터리 교체 비용은 33,500원입니다. 배터리 문제 외에도 다른 원인이 있을 수 있으니, 서비스 센터에 방문하셔서 정확한 진단을 받아보시는 것을 추천드립니다. 추가적인 질문이나 도움이 필요하시면 언제든지 말씀해 주세요!
